In [1]:
import numpy as np
from fractions import gcd
from numbers import Number
import torch
from torch import nn, Tensor
from torch.nn import functional as F
from typing import Any, Callable, Dict, List, Optional, Tuple, Type, Union
from utils import to_long, gpu
import os
from torch.utils.data import DataLoader, Dataset
from utils import collate_fn
from net_M1 import pre_gather

config = dict()
config['n_actornet'] = 128
config['num_epochs'] = 100
config['lr'] = 1e-3
config['train_split'] = '/home/avt/prediction/Waymo/data_processed/train'
config['val_split'] = '/home/avt/prediction/Waymo/data_processed/validation'
config["num_scales"] = 6
config["n_map"] = 128
config["n_actor"] = 128
config["actor2map_dist"] = 7.0
config["map2actor_dist"] = 6.0
config["actor2actor_dist"] = 100.0
config["num_mods"] = 6
config["pred_size"] = 80
config["pred_step"] = 1
config["num_preds"] = config["pred_size"] // config["pred_step"]
config["cls_th"] = 2.0
config["cls_ignore"] = 0.2
config["mgn"] = 0.2
config["cls_coef"] = 1.0
config["reg_coef"] = 1.0

In [ ]:
class W_Dataset(Dataset):
    def __init__(self,path) -> None:

        self.path = path
        self.files = os.listdir(path)
    
    def __getitem__(self, index) -> dict:

        data_path = os.path.join(self.path,self.files[index])
        data = torch.load(data_path)

        return data
    
    def __len__(self) -> int:

        return len(self.files)

In [ ]:
batch_size = 4
dataset_train = W_Dataset(config['train_split'])
train_loader = DataLoader(dataset_train, 
                        batch_size = batch_size ,
                        collate_fn = collate_fn, 
                        shuffle = True, 
                        drop_last=True)

In [ ]:
for batch_idx, data in enumerate(train_loader):
    if batch_idx > 0:
        break
    zz = data

In [ ]:
a = torch.randint(0,2,size =(4,3),dtype=torch.bool)
z = torch.rand(4,3,2)
a,z

In [ ]:
z[a]

In [ ]:
gt = pre_gather(zz['gt2_preds'])
hs = pre_gather(zz['has_preds'])

In [ ]:
gt.size(), hs.size()

In [ ]:
gt[hs].size()

In [23]:
loss_fn = nn.SmoothL1Loss(reduction = 'mean')
loss1 = nn.MSELoss(reduction = 'sum')
out = torch.ones((5,2))*2
gt = torch.zeros((5,2))

In [24]:
out,gt

(tensor([[2., 2.],
         [2., 2.],
         [2., 2.],
         [2., 2.],
         [2., 2.]]),
 tensor([[0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.],
         [0., 0.]]))

In [25]:
loss = loss1(out,gt)
loss

tensor(40.)